In [1]:
!pip install tensorflow==2.16.1
!pip install segmentation-models
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models
!PYTHONHASHSEED=0
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from time import time
import numpy as np
import random as python_random
np.random.seed(0)
python_random.seed(0)
tf.random.set_seed(0)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    F

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
train_dataset_path = '/content/gdrive/MyDrive/Colab Notebooks/Progetto DeepL/train_imgs_2D (1).npy'
train_mask_path = '/content/gdrive/MyDrive/Colab Notebooks/Progetto DeepL/train_labels_2D (1).npy'
val_dataset_path = '/content/gdrive/MyDrive/Colab Notebooks/Progetto DeepL/validation_imgs_2D (1).npy'
val_mask_path = '/content/gdrive/MyDrive/Colab Notebooks/Progetto DeepL/validation_labels_2D (1).npy'

X_train = np.load(train_dataset_path)
Y_train = np.load(train_mask_path)
X_val = np.load(val_dataset_path)
Y_val = np.load(val_mask_path)

In [4]:
!pip install --upgrade keras-cv tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 14.3 MB/s eta 0:00:00


In [5]:
import keras_cv

backbone = keras_cv.models.ResNet50V2Backbone(input_shape=[128, 128, 4])
model = keras_cv.models.segmentation.DeepLabV3Plus(
    num_classes=4, backbone=backbone,
)

100%|██████████| 871/871 [00:00<00:00, 2.16MB/s]


In [6]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=segmentation_models.losses.DiceLoss(), metrics=[segmentation_models.metrics.FScore(), segmentation_models.metrics.IOUScore()])

In [7]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
             ]

In [8]:
history = model.fit(x=X_train, y= Y_train, epochs=30, batch_size=32, verbose=1, validation_data=(X_val, Y_val), callbacks=[callbacks])

Epoch 1/30
116/116 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - f1-score: 0.5098 - iou_score: 0.3957 - loss: 0.4902 - val_f1-score: 0.2791 - val_iou_score: 0.2318 - val_loss: 0.7208
Epoch 2/30
116/116 ━━━━━━━━━━━━━━━━━━━━ 124s 890ms/step - f1-score: 0.7355 - iou_score: 0.6076 - loss: 0.2645 - val_f1-score: 0.5433 - val_iou_score: 0.4534 - val_loss: 0.4566
Epoch 3/30
116/116 ━━━━━━━━━━━━━━━━━━━━ 141s 886ms/step - f1-score: 0.7766 - iou_score: 0.6550 - loss: 0.2234 - val_f1-score: 0.5846 - val_iou_score: 0.4941 - val_loss: 0.4154
Epoch 4/30
116/116 ━━━━━━━━━━━━━━━━━━━━ 142s 888ms/step - f1-score: 0.7965 - iou_score: 0.6795 - loss: 0.2035 - val_f1-score: 0.6013 - val_iou_score: 0.5189 - val_loss: 0.3986
Epoch 5/30
116/116 ━━━━━━━━━━━━━━━━━━━━ 137s 846ms/step - f1-score: 0.8091 - iou_score: 0.6955 - loss: 0.1909 - val_f1-score: 0.5699 - val_iou_score: 0.4785 - val_loss: 0.4300
Epoch 6/30
116/116 ━━━━━━━━━━━━━━━━━━━━ 98s 847ms/step - f1-score: 0.8145 - iou_score: 0.7022 - loss: 0.1855 - val_f1-score

In [9]:
import pandas as pd
epochs_table = pd.DataFrame()
epochs_table['Loss'] = history.history['loss']
epochs_table['Validation Loss'] = history.history['val_loss']
epochs_table['Jaccard'] = history.history['iou_score']
epochs_table['Validation Jaccard'] = history.history['iou_score']
epochs_table['Dice score'] = history.history['f1-score']
epochs_table['Validation Dice score'] = history.history['val_f1-score']

epochs_table
epochs_table.to_csv('train_epochs.csv')

In [10]:
model.save('/content/gdrive/MyDrive/Colab Notebooks/Progetto DeepL/models/model_deeplabv3.h5')
model.save_weights('/content/gdrive/MyDrive/Colab Notebooks/Progetto DeepL/models/model_deeplabv3.weights.h5')